In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 0s (47.8 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 122518 files and dir

In [ ]:
import warnings
warnings.filterwarnings( 'ignore' )

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
if os.name != 'posix':
    plt.rc("font", family="Malgun Gothic")
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score
plt.rc('font', family='NanumBarunGothic') 
from sklearn.utils import class_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sk_label=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/SK_label(5일추가).csv')

In [ ]:
sk=sk_label.drop(['year','month','day','day1_label','day3_label','day5_label'],axis=1)

In [ ]:
sk

,날짜,종가,대비,등락률,거래량,거래대금,시가총액,외국인 보유수량,외국인 지분율,토론방,...,경제심리지수(순환변동치),산업생산지수,물가상승률,소비자신뢰지수,소비자심리지수,경제활동참가율(%),실업률(%),고용률(%),3일 등락률,5일 등락률
0,2017-07-11,69000.0,1700.0,2.53,3187332.0,2.176240e+11,5.023216e+13,365847157.0,50.25,303,...,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,1.32,4.86
1,2017-07-12,68900.0,-100.0,-0.14,3462150.0,2.386036e+11,5.015936e+13,366085238.0,50.29,191,...,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,3.45,1.32
2,2017-07-13,70600.0,1700.0,2.47,5432312.0,3.841257e+11,5.139697e+13,365657970.0,50.23,521,...,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,4.90,3.67
3,2017-07-14,71100.0,500.0,0.71,2931832.0,2.090761e+11,5.176097e+13,365984568.0,50.27,235,...,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,3.04,6.76
4,2017-07-17,71200.0,100.0,0.14,2804598.0,1.990677e+11,5.183377e+13,365314000.0,50.18,235,...,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,3.34,5.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,2023-03-27,85500.0,-1800.0,-2.06,3211190.0,2.751969e+11,6.224420e+13,365466692.0,50.20,84,...,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,-1.61,2.15
1405,2023-03-28,88400.0,2900.0,3.39,3180431.0,2.779770e+11,6.435541e+13,365123199.0,50.15,69,...,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,-0.11,5.74
1406,2023-03-29,86900.0,-1500.0,-1.70,3070422.0,2.675525e+11,6.326341e+13,365150714.0,50.16,90,...,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,-0.46,0.00
1407,2023-03-30,88800.0,1900.0,2.19,4264354.0,3.799672e+11,6.464661e+13,364441191.0,50.06,141,...,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,3.86,0.34


In [ ]:
sk_label['day5_label']

0       maintain
1       maintain
2       maintain
3            buy
4            buy
          ...   
1404    maintain
1405         buy
1406    maintain
1407    maintain
1408    maintain
Name: day5_label, Length: 1409, dtype: object

In [ ]:
label=pd.DataFrame()
#더미변수
# label['buy']=sk['5일 등락률'].apply(lambda x : 1 if x >=5 else 0)
# label['sell']=sk['5일 등락률'].apply(lambda x : 1 if x <=-5 else 0)
#encoding변수
label['encoding'] = sk['5일 등락률'].apply(lambda x: 1 if abs(x) < 5 else 0 if x >= 5 else 2)
#회귀로 예측시 내일 등락률
# label['등락률1']=sk['5일 등락률']

In [ ]:
print('매수 encoding 갯수 : ' ,(label['encoding']==0).sum())
print('유지 encoding 갯수 : ' ,(label['encoding']==1).sum())
print('매도 encoding 갯수 : ' ,(label['encoding']==2).sum())

매수 encoding 갯수 :  222
유지 encoding 갯수 :  997
매도 encoding 갯수 :  190


In [ ]:
sk1=sk.shift(1) # 한 행씩 데이터 아래로 내리기
df=pd.concat([sk1,label],axis=1)
df=df.drop(0,axis=0)
df=df.reset_index(drop=True)

In [ ]:
df

,날짜,종가,대비,등락률,거래량,거래대금,시가총액,외국인 보유수량,외국인 지분율,토론방,...,산업생산지수,물가상승률,소비자신뢰지수,소비자심리지수,경제활동참가율(%),실업률(%),고용률(%),3일 등락률,5일 등락률,encoding
0,2017-07-11,69000.0,1700.0,2.53,3187332.0,2.176240e+11,5.023216e+13,365847157.0,50.25,303.0,...,104.5,97.499,111.0,110.9,63.8,3.4,61.6,1.32,4.86,1
1,2017-07-12,68900.0,-100.0,-0.14,3462150.0,2.386036e+11,5.015936e+13,366085238.0,50.29,191.0,...,104.5,97.499,111.0,110.9,63.8,3.4,61.6,3.45,1.32,1
2,2017-07-13,70600.0,1700.0,2.47,5432312.0,3.841257e+11,5.139697e+13,365657970.0,50.23,521.0,...,104.5,97.499,111.0,110.9,63.8,3.4,61.6,4.90,3.67,0
3,2017-07-14,71100.0,500.0,0.71,2931832.0,2.090761e+11,5.176097e+13,365984568.0,50.27,235.0,...,104.5,97.499,111.0,110.9,63.8,3.4,61.6,3.04,6.76,0
4,2017-07-17,71200.0,100.0,0.14,2804598.0,1.990677e+11,5.183377e+13,365314000.0,50.18,235.0,...,104.5,97.499,111.0,110.9,63.8,3.4,61.6,3.34,5.79,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,2023-03-24,87300.0,-1200.0,-1.36,4670908.0,4.121193e+11,6.355461e+13,366220750.0,50.30,155.0,...,112.5,110.560,92.0,92.0,64.1,2.9,62.2,4.43,3.93,1
1404,2023-03-27,85500.0,-1800.0,-2.06,3211190.0,2.751969e+11,6.224420e+13,365466692.0,50.20,84.0,...,112.5,110.560,92.0,92.0,64.1,2.9,62.2,-1.61,2.15,0
1405,2023-03-28,88400.0,2900.0,3.39,3180431.0,2.779770e+11,6.435541e+13,365123199.0,50.15,69.0,...,112.5,110.560,92.0,92.0,64.1,2.9,62.2,-0.11,5.74,1
1406,2023-03-29,86900.0,-1500.0,-1.70,3070422.0,2.675525e+11,6.326341e+13,365150714.0,50.16,90.0,...,112.5,110.560,92.0,92.0,64.1,2.9,62.2,-0.46,0.00,1


In [ ]:
df1=df

In [ ]:
#full모델, 변수선택한 모델 용으로 각각 list만들었습니다.
full_list = ['종가', '대비', '등락률', '거래량', '거래대금', '시가총액',
       '외국인 보유수량', '외국인 지분율', '토론방', '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인',
       '검색어', '보도량', '기사감성점수', '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동',
       '코스피종가', '코스피등락률', '코스피거래량', '코스피거래대금', '코스피시가총액', '한은금리',
       '원/미국달러', '원/위안', '원/일본엔(100엔)', '원/유로', '경제심리지수(원계열)', '경제심리지수(순환변동치)',
       '산업생산지수', '물가상승률', '소비자신뢰지수', '소비자심리지수', '경제활동참가율(%)', '실업률(%)',
       '고용률(%)','코스피대비']
cor_list=['등락률','토론방','순매수_기관','순매수_기타법인','순매수_개인','순매수_외국인','기사감성점수','코스피등락률','대비'] #1일치
cor_list3=['등락률','종가','거래량','거래대금','시가총액','토론방','순매수_기관','순매수_기타법인','순매수_개인','순매수_외국인','검색어','보도량','기사감성점수','코스피등락률','코스피거래대금'] #3일치
vif_list=['경제심리지수(순환변동치)','시가총액','비트코인종가','원/미국달러','소비자심리지수','코스피거래대금','순매수_개인',
         '순매수_외국인','산업생산지수','코스피거래량','뉴스심리지수','원/유로','토론방','실업률(%)','원/일본엔(100엔)',
    '거래량','기사감성점수','외국인 보유수량','코스피등락률','경제활동참가율(%)','검색어','보도량','순매수_기관',
    '비트코인거래량','비트코인변동']

In [ ]:
df1.columns

Index(['날짜', '종가', '대비', '등락률', '거래량', '거래대금', '시가총액', '외국인 보유수량', '외국인 지분율',
       '토론방', '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인', '검색어', '보도량',
       '기사감성점수', '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동', '코스피종가', '코스피대비',
       '코스피등락률', '코스피거래량', '코스피거래대금', '코스피시가총액', '한은금리', '원/미국달러', '원/위안',
       '원/일본엔(100엔)', '원/유로', '경제심리지수(원계열)', '경제심리지수(순환변동치)', '산업생산지수',
       '물가상승률', '소비자신뢰지수', '소비자심리지수', '경제활동참가율(%)', '실업률(%)', '고용률(%)',
       '3일 등락률', '5일 등락률', 'encoding'],
      dtype='object')

In [ ]:
# 더미변수로 진행할꺼면 y=df1[['buy','sell']]
# encoding으로 진행할꺼면 y=df1['encoding']
# 회귀로 진행할꺼면 y=df1['등락률1']
x=df1[full_list]
y=df1['encoding']

In [ ]:
x

,종가,대비,등락률,거래량,거래대금,시가총액,외국인 보유수량,외국인 지분율,토론방,순매수_기관,...,경제심리지수(원계열),경제심리지수(순환변동치),산업생산지수,물가상승률,소비자신뢰지수,소비자심리지수,경제활동참가율(%),실업률(%),고용률(%),코스피대비
0,69000.0,1700.0,2.53,3187332.0,2.176240e+11,5.023216e+13,365847157.0,50.25,303.0,1.812985e+10,...,99.1,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,13.90
1,68900.0,-100.0,-0.14,3462150.0,2.386036e+11,5.015936e+13,366085238.0,50.29,191.0,-9.839816e+09,...,99.1,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,-4.23
2,70600.0,1700.0,2.47,5432312.0,3.841257e+11,5.139697e+13,365657970.0,50.23,521.0,-5.082932e+09,...,99.1,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,17.72
3,71100.0,500.0,0.71,2931832.0,2.090761e+11,5.176097e+13,365984568.0,50.27,235.0,1.049480e+10,...,99.1,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,5.14
4,71200.0,100.0,0.14,2804598.0,1.990677e+11,5.183377e+13,365314000.0,50.18,235.0,6.822217e+09,...,99.1,100.0,104.5,97.499,111.0,110.9,63.8,3.4,61.6,10.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,87300.0,-1200.0,-1.36,4670908.0,4.121193e+11,6.355461e+13,366220750.0,50.30,155.0,2.809119e+10,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,-9.52
1404,85500.0,-1800.0,-2.06,3211190.0,2.751969e+11,6.224420e+13,365466692.0,50.20,84.0,-2.786144e+09,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,-5.74
1405,88400.0,2900.0,3.39,3180431.0,2.779770e+11,6.435541e+13,365123199.0,50.15,69.0,5.638837e+10,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,25.72
1406,86900.0,-1500.0,-1.70,3070422.0,2.675525e+11,6.326341e+13,365150714.0,50.16,90.0,2.506547e+10,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,8.98


In [ ]:
y

0       1
1       1
2       0
3       0
4       1
       ..
1403    1
1404    0
1405    1
1406    1
1407    1
Name: encoding, Length: 1408, dtype: int64

#분류(XGB,LGB)

In [ ]:
#### 엔코딩으로 진행
#### 더미변수 말고 0,1,2로 라벨링하면 이거 쓰시면 됩니다.
tscv=TimeSeriesSplit(n_splits=4)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train=x.iloc[train_index], y.iloc[train_index]
    x_test, y_test=x.iloc[test_index],y.iloc[test_index]
    classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)
    params = {'objective': 'multi:softmax', 'num_class': 3}
    model=LGBMClassifier(**params, random_state=42)
    model.fit(x_train, y_train, sample_weight=classes_weights)
   # model = XGBClassifier(**params).fit(x_train, y_train,sample_weight=classes_weights)
    
    y_pred=model.predict(x_test)
    print(f"Fold {i+1} - accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test,y_pred,average='weighted')}")

Fold 1 - accuracy: 0.6120996441281139
Fold 1 - f1_score: 0.5916795554801143
Fold 2 - accuracy: 0.6476868327402135
Fold 2 - f1_score: 0.6465423170807845
Fold 3 - accuracy: 0.6227758007117438
Fold 3 - f1_score: 0.6241800147304805
Fold 4 - accuracy: 0.6370106761565836
Fold 4 - f1_score: 0.6554073499980329


In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 25,  12,   0],
       [ 46, 132,  20],
       [  5,  19,  22]])

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('매수 정확도 : ',cm[0,0]/sum(cm[0]))
print('매도 정확도 : ',cm[2,2]/sum(cm[2]))
print(cm)

매수 정확도 :  0.6756756756756757
매도 정확도 :  0.4782608695652174
[[ 25  12   0]
 [ 46 132  20]
 [  5  19  22]]


# LGBM f1score로 optuna

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import optuna
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 20, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 2, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 1e-8, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 1e-8, 1.0)
    }

    scores = []
    for i, (train_index, test_index) in enumerate(tscv.split(x)):
        x_train, y_train=x.iloc[train_index], y.iloc[train_index]
        x_test, y_test=x.iloc[test_index], y.iloc[test_index]

        classes_weights = class_weight.compute_sample_weight(
            class_weight='balanced', y=y_train)

        lgbm_model=LGBMClassifier(**params, boost_from_average=False,
                                  application='multiclass', is_unbalance=True,
                                  random_state=42)
        lgbm_model.fit(x_train, y_train, sample_weight=classes_weights)

        y_pred=model.predict(x_test)
        score = f1_score(y_test, y_pred, average='weighted')
        scores.append(score)

    return sum(scores) / len(scores)


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2023-05-07 11:23:43,254] A new study created in memory with name: no-name-51f0427a-f2a4-4c8e-b123-e3ac18fdceeb
[I 2023-05-07 11:23:47,837] Trial 0 finished with value: 0.577398181739923 and parameters: {'n_estimators': 180, 'max_depth': 15, 'learning_rate': 0.03005641375140659, 'num_leaves': 36, 'min_child_samples': 62, 'subsample': 0.04744359198374499, 'colsample_bytree': 0.9911251145512401, 'reg_alpha': 0.06394068771750928, 'reg_lambda': 0.28062772590188584}. Best is trial 0 with value: 0.577398181739923.
[I 2023-05-07 11:24:04,140] Trial 1 finished with value: 0.6172303292703527 and parameters: {'n_estimators': 1487, 'max_depth': 8, 'learning_rate': 0.053291753278004214, 'num_leaves': 45, 'min_child_samples': 94, 'subsample': 0.19922041312305389, 'colsample_bytree': 0.37020949448800244, 'reg_alpha': 0.002635097750486308, 'reg_lambda': 0.23164402346710047}. Best is trial 1 with value: 0.6172303292703527.
[I 2023-05-07 11:24:10,419] Trial 2 finished with value: 0.6143319491263975 a

In [ ]:
print(study.best_value)
print(study.best_params)

0.6465398931582731
{'n_estimators': 340, 'max_depth': 19, 'learning_rate': 0.010081489110639592, 'num_leaves': 36, 'min_child_samples': 18, 'subsample': 0.539172634939373, 'colsample_bytree': 0.07593962375940377, 'reg_alpha': 0.07618155673873425, 'reg_lambda': 0.3443848988819511}


In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=4)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x.iloc[test_index], y.iloc[test_index]

    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced', y=y_train)
    
    model=LGBMClassifier(**study.best_params, boost_from_average=False,
                             application='multiclass', is_unbalance=True,
                             random_state=42)
    model.fit(x_train, y_train, sample_weight=classes_weights)

    y_pred = model.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.6581196581196581
Fold 1 - f1_score: 0.5796771130104464

Fold 2 - Accuracy: 0.6367521367521367
Fold 2 - f1_score: 0.6262569364714886

Fold 3 - Accuracy: 0.5726495726495726
Fold 3 - f1_score: 0.6099334001310498

Fold 4 - Accuracy: 0.7649572649572649
Fold 4 - f1_score: 0.7435049034417544

Fold 5 - Accuracy: 0.7094017094017094
Fold 5 - f1_score: 0.6733271127366264



In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 11,  20,   1],
       [ 11, 147,   5],
       [  0,  31,   8]])

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('매수 정확도 : ',cm[0,0]/sum(cm[0]))
print('매도 정확도 : ',cm[2,2]/sum(cm[2]))
print(cm)

매수 정확도 :  0.34375
매도 정확도 :  0.20512820512820512
[[ 11  20   1]
 [ 11 147   5]
 [  0  31   8]]


# LGBM 커스텀score로 optuna

매도 매수 정확도의 평균을 return으로 optuna 학습

In [ ]:
import optuna
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 20, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 2, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 1e-8, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 1e-8, 1.0)
    }

    scores = []
    for i, (train_index, test_index) in enumerate(tscv.split(x)):
        x_train, y_train=x.iloc[train_index], y.iloc[train_index]
        x_test, y_test=x.iloc[test_index], y.iloc[test_index]

        classes_weights = class_weight.compute_sample_weight(
            class_weight='balanced', y=y_train)

        lgbm_model=LGBMClassifier(**params, boost_from_average=False,
                                  application='multiclass', is_unbalance=True,
                                  random_state=42)
        lgbm_model.fit(x_train, y_train, sample_weight=classes_weights)

        y_pred=lgbm_model.predict(x_test)
        score = f1_score(y_test, y_pred, average='weighted')
        scores.append(score)

    cm = confusion_matrix(y_test, y_pred)
    bacc = cm[0,0]/sum(cm[0]) #매수 정확도
    sacc = cm[2,2]/sum(cm[2]) #매도 정확도
    rst = np.mean([bacc, sacc])

    return rst

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2023-05-07 11:55:02,617] A new study created in memory with name: no-name-2c06235a-e907-4c15-820e-dbbd6aa86f52
[I 2023-05-07 11:55:12,456] Trial 0 finished with value: 0.3000801282051282 and parameters: {'n_estimators': 1121, 'max_depth': 16, 'learning_rate': 0.022505248791912003, 'num_leaves': 48, 'min_child_samples': 46, 'subsample': 0.1945291131460817, 'colsample_bytree': 0.17871082861657125, 'reg_alpha': 0.6263218157922401, 'reg_lambda': 0.0011333725701622522}. Best is trial 0 with value: 0.3000801282051282.
[I 2023-05-07 11:55:15,454] Trial 1 finished with value: 0.6129807692307692 and parameters: {'n_estimators': 872, 'max_depth': 15, 'learning_rate': 0.0011112731413034289, 'num_leaves': 8, 'min_child_samples': 53, 'subsample': 0.46231819395261786, 'colsample_bytree': 0.07052904517856191, 'reg_alpha': 0.6059832312352418, 'reg_lambda': 0.44229475222373815}. Best is trial 1 with value: 0.6129807692307692.
[I 2023-05-07 11:55:43,656] Trial 2 finished with value: 0.657051282051282

In [ ]:
print(study.best_value)
print(study.best_params)

0.7936698717948718
{'n_estimators': 60, 'max_depth': 18, 'learning_rate': 0.0006132003558847834, 'num_leaves': 2, 'min_child_samples': 60, 'subsample': 0.7366907111512055, 'colsample_bytree': 0.5073608380038124, 'reg_alpha': 0.10982459464284575, 'reg_lambda': 0.7573655695836168}


In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=4)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x.iloc[test_index], y.iloc[test_index]

    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced', y=y_train)
    
    model=LGBMClassifier(**study.best_params, boost_from_average=False,
                             application='multiclass', is_unbalance=True,
                             random_state=42)
    model.fit(x_train, y_train, sample_weight=classes_weights)

    y_pred = model.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.5641025641025641
Fold 1 - f1_score: 0.5473878258628437

Fold 2 - Accuracy: 0.5085470085470085
Fold 2 - f1_score: 0.5426155841451062

Fold 3 - Accuracy: 0.3247863247863248
Fold 3 - f1_score: 0.322989991843824

Fold 4 - Accuracy: 0.2863247863247863
Fold 4 - f1_score: 0.2878103378103378

Fold 5 - Accuracy: 0.3717948717948718
Fold 5 - f1_score: 0.34450332521398513



In [ ]:
confusion_matrix(y_test, y_pred)

array([[27,  2,  3],
       [58, 31, 74],
       [ 9,  1, 29]])

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('매수 정확도 : ',cm[0,0]/sum(cm[0]))
print('매도 정확도 : ',cm[2,2]/sum(cm[2]))
print(cm)

매수 정확도 :  0.84375
매도 정확도 :  0.7435897435897436
[[27  2  3]
 [58 31 74]
 [ 9  1 29]]


# LGBM 커스텀score2 optuna

In [ ]:
import optuna
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 20, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 2, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 1e-8, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 1e-8, 1.0)
    }

    scores = []
    for i, (train_index, test_index) in enumerate(tscv.split(x)):
        x_train, y_train=x.iloc[train_index], y.iloc[train_index]
        x_test, y_test=x.iloc[test_index], y.iloc[test_index]

        classes_weights = class_weight.compute_sample_weight(
            class_weight='balanced', y=y_train)

        lgbm_model=LGBMClassifier(**params, boost_from_average=False,
                                  application='multiclass', is_unbalance=True,
                                  random_state=42)
        lgbm_model.fit(x_train, y_train, sample_weight=classes_weights)

        y_pred=lgbm_model.predict(x_test)
        score = f1_score(y_test, y_pred, average='weighted')
        scores.append(score)

    cm = confusion_matrix(y_test, y_pred)
    bacc = cm[0,0]/sum(cm[0]) #매수 정확도
    sacc = cm[2,2]/sum(cm[2]) #매도 정확도
    f1 = sum(scores) / len(scores) 
    rst = np.mean([bacc, sacc, f1])

    return rst

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2023-05-07 12:10:16,251] A new study created in memory with name: no-name-f9375da0-535a-4a16-928a-edd8b32cb7b0
[I 2023-05-07 12:10:47,121] Trial 0 finished with value: 0.5689335556503445 and parameters: {'n_estimators': 1267, 'max_depth': 13, 'learning_rate': 0.0010354831823572762, 'num_leaves': 23, 'min_child_samples': 8, 'subsample': 0.7346553563165527, 'colsample_bytree': 0.3953295656001824, 'reg_alpha': 0.453103689073909, 'reg_lambda': 0.855629131455626}. Best is trial 0 with value: 0.5689335556503445.
[I 2023-05-07 12:10:51,487] Trial 1 finished with value: 0.34695881433389747 and parameters: {'n_estimators': 1277, 'max_depth': 10, 'learning_rate': 0.25926223186840197, 'num_leaves': 20, 'min_child_samples': 16, 'subsample': 0.9939948767439395, 'colsample_bytree': 0.8905019368970638, 'reg_alpha': 0.20095099359254917, 'reg_lambda': 0.10323632511208666}. Best is trial 0 with value: 0.5689335556503445.
[I 2023-05-07 12:10:57,831] Trial 2 finished with value: 0.45651977268370203 and

In [ ]:
print(study.best_value)
print(study.best_params)

0.6772643244292648
{'n_estimators': 1520, 'max_depth': 19, 'learning_rate': 0.0006656085775987695, 'num_leaves': 21, 'min_child_samples': 92, 'subsample': 0.11098219181358777, 'colsample_bytree': 0.5216102348994016, 'reg_alpha': 0.8695265426016873, 'reg_lambda': 0.2567638323301277}


In [ ]:
tscv=TimeSeriesSplit(n_splits=4)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x.iloc[test_index], y.iloc[test_index]

    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced', y=y_train)
    
    model=LGBMClassifier(**study.best_params, boost_from_average=False,
                             application='multiclass', is_unbalance=True,
                             random_state=42)
    model.fit(x_train, y_train, sample_weight=classes_weights)

    y_pred = model.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.4306049822064057
Fold 1 - f1_score: 0.44494288352764444

Fold 2 - Accuracy: 0.4412811387900356
Fold 2 - f1_score: 0.46266963052636334

Fold 3 - Accuracy: 0.4199288256227758
Fold 3 - f1_score: 0.4617839807076067

Fold 4 - Accuracy: 0.5836298932384342
Fold 4 - f1_score: 0.6075030734390167



In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 28,   6,   3],
       [ 40, 109,  49],
       [  5,  14,  27]])

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('매수 정확도 : ',cm[0,0]/sum(cm[0]))
print('매도 정확도 : ',cm[2,2]/sum(cm[2]))
print('유지 정확도 : ',cm[1,1]/sum(cm[1]))

매수 정확도 :  0.7567567567567568
매도 정확도 :  0.5869565217391305
유지 정확도 :  0.5505050505050505


In [ ]:
np.sum(cm, axis=1), np.sum(cm, axis=0)

(array([ 37, 198,  46]), array([ 73, 129,  79]))

In [ ]:
cm[0,0]/np.sum(cm, axis=0)[0]

0.3835616438356164

# LGBM 커스텀score3 optuna

매도매수의 정확도, 정밀도를 score로

In [ ]:
import optuna
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 20, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 0.5),
        'num_leaves': trial.suggest_int('num_leaves', 2, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 1e-8, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 1e-8, 1.0)
    }

    scores = []
    for i, (train_index, test_index) in enumerate(tscv.split(x)):
        x_train, y_train=x.iloc[train_index], y.iloc[train_index]
        x_test, y_test=x.iloc[test_index], y.iloc[test_index]

        classes_weights = class_weight.compute_sample_weight(
            class_weight='balanced', y=y_train)

        lgbm_model=LGBMClassifier(**params, boost_from_average=False,
                                  application='multiclass', is_unbalance=True,
                                  random_state=42)
        lgbm_model.fit(x_train, y_train, sample_weight=classes_weights)

        y_pred=lgbm_model.predict(x_test)
        score = f1_score(y_test, y_pred, average='weighted')
        scores.append(score)

    cm = confusion_matrix(y_test, y_pred)
    bacc = cm[0,0]/sum(cm[0]) #매수 정확도
    sacc = cm[2,2]/sum(cm[2]) #매도 정확도
    bpre = cm[0,0]/np.sum(cm, axis=0)[0] #매수 정밀도
    spre = cm[2,2]/np.sum(cm, axis=0)[2] #매도 정밀도
    rst = np.mean([bacc, sacc, bpre, spre])

    return rst

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2023-05-07 12:37:33,100] A new study created in memory with name: no-name-dba3f25a-c296-47e1-b66e-8aaa46300be9
[I 2023-05-07 12:37:33,558] Trial 0 finished with value: 0.37588249306999305 and parameters: {'n_estimators': 73, 'max_depth': 10, 'learning_rate': 0.03986574872892966, 'num_leaves': 22, 'min_child_samples': 33, 'subsample': 0.4979893614083614, 'colsample_bytree': 0.048422876126084584, 'reg_alpha': 0.4245352904918848, 'reg_lambda': 0.23156678843541012}. Best is trial 0 with value: 0.37588249306999305.
[I 2023-05-07 12:37:39,248] Trial 1 finished with value: 0.26130651147098516 and parameters: {'n_estimators': 1184, 'max_depth': 15, 'learning_rate': 0.38893687654524606, 'num_leaves': 7, 'min_child_samples': 27, 'subsample': 0.7829939860029496, 'colsample_bytree': 0.5507502169765977, 'reg_alpha': 0.911438967187885, 'reg_lambda': 0.08588752481047832}. Best is trial 0 with value: 0.37588249306999305.
[I 2023-05-07 12:37:46,280] Trial 2 finished with value: 0.5280971293199554 an

In [ ]:
print(study.best_value)
print(study.best_params)

0.5968258891645989
{'n_estimators': 81, 'max_depth': 5, 'learning_rate': 0.00017562529067599212, 'num_leaves': 10, 'min_child_samples': 78, 'subsample': 0.06675720428336028, 'colsample_bytree': 0.7253157173071396, 'reg_alpha': 0.1548899107799841, 'reg_lambda': 0.525179675965547}


In [ ]:
tscv=TimeSeriesSplit(n_splits=4)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x.iloc[test_index], y.iloc[test_index]

    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced', y=y_train)
    
    model=LGBMClassifier(**study.best_params, boost_from_average=False,
                             application='multiclass', is_unbalance=True,
                             random_state=42)
    model.fit(x_train, y_train, sample_weight=classes_weights)

    y_pred = model.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.24199288256227758
Fold 1 - f1_score: 0.15865005924015518

Fold 2 - Accuracy: 0.4199288256227758
Fold 2 - f1_score: 0.4416787134595453

Fold 3 - Accuracy: 0.4234875444839858
Fold 3 - f1_score: 0.4683646297779495

Fold 4 - Accuracy: 0.5800711743772242
Fold 4 - f1_score: 0.605502467437996



In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 24,   9,   4],
       [ 37, 112,  49],
       [  5,  14,  27]])

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('전체 정확도 : ', accuracy_score(y_test, y_pred))
print('전체 f1-score : ', f1_score(y_test, y_pred, average='weighted'))
print()
print('매수 정확도 : ', cm[0,0]/sum(cm[0]))
print('매도 정확도 : ', cm[2,2]/sum(cm[2]))
print('유지 정확도 : ', cm[1,1]/sum(cm[1]))
print()
print('매수 정밀도 : ', cm[0,0]/np.sum(cm, axis=0)[0])
print('매도 정밀도 : ', cm[2,2]/np.sum(cm, axis=0)[2])

전체 정확도 :  0.5800711743772242
전체 f1-score :  0.605502467437996

매수 정확도 :  0.6486486486486487
매도 정확도 :  0.5869565217391305
유지 정확도 :  0.5656565656565656

매수 정밀도 :  0.36363636363636365
매도 정밀도 :  0.3375
